In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')

In [4]:
data.head(10)

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
5,1619387,What should I do if I know someone who appears...,Although this website cannot substitute for pr...
6,1030153,How can I find a mental health professional fo...,Feeling comfortable with the professional you ...
7,8022026,What treatment options are available?,Just as there are different types of medicatio...
8,1155199,"If I become involved in treatment, what do I n...",Since beginning treatment is a big step for in...
9,7760466,What is the difference between mental health p...,There are many types of mental health professi...


In [5]:
data.tail(10)

,Question_ID,Questions,Answers
88,1896541,What are positive and negative symptoms?,Positive and negative symptoms are medical ter...
89,4962901,What is a prodrome?,Prodrome is a medical term for early signs or ...
90,6869967,I’m an adult and I think I might have ADHD. Wh...,While we think of attention-deficit/hyperactiv...
91,4824231,What’s the difference between substance use an...,People tend to use the word “addiction” to mea...
92,4294616,How can I find help for an alcohol or drug use...,Seeking to make changes in the way you use sub...
93,4373204,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,7807643,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,4352464,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,6521784,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...
97,3221856,What is the evidence on vaping?,"""Vaping"" is the term for using a device where ..."


# Fix this first question with text to text generation

# What causes mental illness?

In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "Mental illnesses are health conditions that disrupt a persona's thoughts, emotions, relationships, ..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Original Question: Mental illnesses are health conditions that disrupt a persona's thoughts, emotions, relationships, ...
Paraphrases:
1. Health issues that impact a person's thinking, feeling, relationships, and other aspects are classified as mental illnesses.
2. Mental disorders are mental health conditions that disrupt a person's thought process, emotions, relationships, or symptoms.
3. Disorders characterized by mental health disorders that disrupt a person's thinking, feelings, relationships, and other vital faculties.
4. A person's mental health condition, or mental illness, can disrupt their thought process/emotions, relationships, and more.
5. Psychiatric disorders are conditions that affect an individual's thinking, feelings and relationships.


# Use model deepset/roberta-base-squad2 to train dataset

In [23]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "What causes mental illness?"
context = "Health issues that impact a person's thinking, feeling, relationships, and other aspects are classified as mental illnesses."
qa_model(question=question, context=context)

{'score': 0.7215068936347961, 'start': 0, 'end': 13, 'answer': 'Health issues'}

# What are some of the warning signs of mental illness?

In [24]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "Symptoms of mental health disorders vary depending on the type and severity of the condition. The fo..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: Symptoms of mental health disorders vary depending on the type and severity of the condition. The fo...
Paraphrases:
1. The symptoms of mental health disorders are influenced by the type and severity of the condition.
2. Symptoms of mental health disorders are distinct, depending on the type and severity of the condition.
3. symptoms of mental health disorders are determined by the type and severity of the condition, if any at all.
4. mental health disorders can manifest in a variety of ways, depending on the condition and level of severity.
5. Mental health conditions manifest in a variety of ways, with symptoms varying depending on the disorder and its intensity.


In [25]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "What are some of the warning signs of mental illness?"
context = "The symptoms of mental health disorders are influenced by the type and severity of the condition."
qa_model(question=question, context=context)

{'score': 0.2266746163368225,
 'start': 62,
 'end': 79,
 'answer': 'type and severity'}

# Can people with mental illness recover?

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "When healing from mental illness, early identification and treatment are of vital importance. Based ..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: When healing from mental illness, early identification and treatment are of vital importance. Based ...
Paraphrases:
1. Mental illness recovery requires early identification and treatment.
2. It is crucial to identify and treat mental illness as early as possible when it affects one's recovery.
3. The identification and treatment of mental illness is a crucial aspect of recovery.
4. Identifying and treating mental illnesses early is crucial in the process of healing.
5. Early detection and rehabilitation are crucial in the treatment of mental illness during healing.


In [27]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "Can people with mental illness recover?"
context = "Identifying and treating mental illnesses early is crucial in the process of healing."
qa_model(question=question, context=context)

{'score': 0.24204491078853607,
 'start': 0,
 'end': 84,
 'answer': 'Identifying and treating mental illnesses early is crucial in the process of healing'}

# What should I do if I know someone who appears to have the symptoms of a mental disorder?

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "Although this website cannot substitute for professional advice, we encourage those with symptoms to..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: Although this website cannot substitute for professional advice, we encourage those with symptoms to...
Paraphrases:
1. This website is not a substitute for professional advice, but we suggest individuals with symptoms to...
2. Although not a substitute for professional guidance, this website advocates for individuals with symptoms to...
3. While this website is not a substitute for professional guidance, we recommend that individuals with symptoms of illness...
4. Despite being an intermediary site, we cannot offer professional guidance, and those experiencing symptoms should...
5. We cannot provide professional advice on this website, but we suggest that those with symptoms seek assistance from...


In [29]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "What should I do if I know someone who appears to have the symptoms of a mental disorder?"
context = "We cannot provide professional advice on this website, but we suggest that those with symptoms seek assistance from certified counselor..."
qa_model(question=question, context=context)

{'score': 0.3380628824234009,
 'start': 95,
 'end': 135,
 'answer': 'seek assistance from certified counselor'}

# How can I find a mental health professional for myself or my child?

In [30]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "Feeling comfortable with the professional you or your child is working with is critical to the succe..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: Feeling comfortable with the professional you or your child is working with is critical to the succe...
Paraphrases:
1. It is essential for the individual to feel at ease with the professional working alongside them...
2. To ensure the comfort of the person working, it is essential to feel at ease with the professional you or your child...
3. Whether you or your child are with the professional, it is essential to feel at ease...
4. The key to the outcome is that you or your child feel at ease with the professional they are working with...
5. It is crucial to feel at ease with the professional you or your child are collaborating with...


In [32]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "How can I find a mental health professional for myself or my child?"
context = "It is essential for the individual to feel at ease with the professional working alongside them..."
qa_model(question=question, context=context)

{'score': 0.0037065150681883097,
 'start': 38,
 'end': 95,
 'answer': 'feel at ease with the professional working alongside them'}

# How can I challenge thinking traps?

In [37]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "What you tell yourself about a situation affects how you feel and what you do. Sometimes your interp."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: What you tell yourself about a situation affects how you feel and what you do. Sometimes your interp.
Paraphrases:
1. The information you impart to yourself about a situation can impact your emotional state and behavior.
2. Your self-talk about a situation can impact your emotional state and behavior.
3. How you approach a situation affects the way you feel and behave, which can be influenced by your self-talk.
4. Depending on the information you impart to yourself about a situation, your emotional state and actions may be influenced by these patterns.
5. The things you say or do can impact your emotional state and behavior.


In [38]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "How can I challenge thinking traps?"
context = "Depending on the information you impart to yourself about a situation, your emotional state and actions may be influenced by these patterns."
qa_model(question=question, context=context)

{'score': 0.0002849921875167638,
 'start': 0,
 'end': 69,
 'answer': 'Depending on the information you impart to yourself about a situation'}

# Where can I find self-help materials for depression?

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Check if GPU is available, if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

# Define the paraphrase function
def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    # Decode the generated outputs
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

# Test the paraphrase function
question = "Self-help resources are not a substitute for diagnosis or treatment. If you have concerns about low ..."
paraphrases = paraphrase(question)
print("Original Question:", question)
print("Paraphrases:")
for idx, paraphrase in enumerate(paraphrases):
    print(f"{idx+1}. {paraphrase}")

Original Question: Self-help resources are not a substitute for diagnosis or treatment. If you have concerns about low ...
Paraphrases:
1. There are no self-help resources that can provide diagnosis or treatment. If you have questions about low...
2. The information provided on self-help resources is not intended to diagnose or treat an illness. If you are experiencing symptoms of low...
3. Diagnosis and treatment are not the same as accessing self-help resources. If you have questions about low...
4. We do not provide diagnosis or treatment for any illness. If you are concerned about low...
5. Self-help resources are not a substitute for diagnosis and treatment. If you experience symptoms of low...


In [35]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")
question = "Where can I find self-help materials for depression?"
context = "There are no self-help resources that can provide diagnosis or treatment. If you have questions about low..."
qa_model(question=question, context=context)

{'score': 0.039537250995635986,
 'start': 0,
 'end': 32,
 'answer': 'There are no self-help resources'}